In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
# import hvplot.pandas
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import datetime as dt

%matplotlib inline

In [3]:
filepath = Path("../data/MSFT_EWMA_MACD_signals.csv")

In [4]:
msft_ewma_macd_df = pd.read_csv(filepath)
msft_ewma_macd_df["timestamp"] = pd.to_datetime(msft_ewma_macd_df["timestamp"]).dt.date
msft_ewma_macd_df = msft_ewma_macd_df.set_index("timestamp")
msft_ewma_macd_df.head()

,Short_EWMA,Long_EWMA,Final_Signal,macd,signal_lag,divergence
timestamp,,,,,,
2017-06-01,70.100000,70.100000,0.0,0.000000,0.000000,0.000000
2017-06-02,70.999167,70.961923,0.0,0.037244,0.020691,0.016553
2017-06-05,71.499076,71.435495,0.0,0.063581,0.038269,0.025312
2017-06-06,71.821342,71.738675,0.0,0.082667,0.053309,0.029358
2017-06-07,71.975844,71.889720,0.0,0.086124,0.063070,0.023054


In [5]:
msft_ewma_macd_df.shape

(1260, 6)

In [6]:
X = msft_ewma_macd_df[["Short_EWMA", "Long_EWMA", "macd", "signal_lag"]]
X = X.drop(X.index[1259])
# display(X.head())
# display(X.tail())
X

,Short_EWMA,Long_EWMA,macd,signal_lag
timestamp,,,,
2017-06-01,70.100000,70.100000,0.000000,0.000000
2017-06-02,70.999167,70.961923,0.037244,0.020691
2017-06-05,71.499076,71.435495,0.063581,0.038269
2017-06-06,71.821342,71.738675,0.082667,0.053309
2017-06-07,71.975844,71.889720,0.086124,0.063070
...,...,...,...,...
2022-05-24,261.857360,269.924942,-8.067582,-7.886178
2022-05-25,261.959304,269.376428,-7.417124,-7.792367
2022-05-26,262.565565,269.118915,-6.553349,-7.544564


In [7]:
y = msft_ewma_macd_df["Final_Signal"]
y = y.shift(1).dropna()
y

timestamp
2017-06-02    0.0
2017-06-05    0.0
2017-06-06    0.0
2017-06-07    0.0
2017-06-08    0.0
             ... 
2022-05-25    0.0
2022-05-26    0.0
2022-05-27    0.0
2022-05-31    0.0
2022-06-01    0.0
Name: Final_Signal, Length: 1259, dtype: float64

In [8]:
# from pandas.tseries.offsets import DateOffset

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [10]:
# training_end = X.index.min() + DateOffset(months= 42) 

# print(training_end)

In [11]:
# X_train = X.loc[training_begin:training_end]
# y_train = y.loc[training_begin:training_end]

In [12]:
# X_test = X.loc[training_end:]
# y_test = y.loc[training_end:]



In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler model to the X training data
X_scaler = scaler.fit(X_train)

# Transform the X_train and X_test Data using the Scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



In [14]:

ann = tf.keras.models.Sequential()

number_units= 100

# Layer 1
ann.add(tf.keras.layers.Dense(units=number_units, activation="sigmoid"))

# Layer 2
ann.add(tf.keras.layers.Dense(units=50, activation="sigmoid"))

# Layer 3
ann.add(tf.keras.layers.Dense(units=40, activation="tanh"))

# Output layer
ann.add(tf.keras.layers.Dense(units=1, activation = "sigmoid"))   

In [15]:
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [16]:
# ann.summary()

In [17]:
ann.fit(X_train_scaled, y_train, epochs=500, batch_size=32)

Epoch 1/500
32/32 [==============================] - 0s 1ms/step - loss: 0.6841 - accuracy: 0.5501
Epoch 2/500
32/32 [==============================] - 0s 1ms/step - loss: 0.6652 - accuracy: 0.5750
Epoch 3/500
32/32 [==============================] - 0s 1ms/step - loss: 0.6474 - accuracy: 0.6097
Epoch 4/500
32/32 [==============================] - 0s 1ms/step - loss: 0.6224 - accuracy: 0.6256
Epoch 5/500
32/32 [==============================] - 0s 1ms/step - loss: 0.6191 - accuracy: 0.6286
Epoch 6/500
32/32 [==============================] - 0s 1ms/step - loss: 0.6049 - accuracy: 0.6435
Epoch 7/500
32/32 [==============================] - 0s 1ms/step - loss: 0.5839 - accuracy: 0.6653
Epoch 8/500
32/32 [==============================] - 0s 1ms/step - loss: 0.5823 - accuracy: 0.6594
Epoch 9/500
32/32 [==============================] - 0s 1ms/step - loss: 0.5744 - accuracy: 0.6822
Epoch 10/500
32/32 [==============================] - 0s 2ms/step - loss: 0.5644 - accuracy: 0.6862
Epoch 11/

In [18]:
model_loss, model_accuracy = ann.evaluate(X_test_scaled, y_test, verbose=2)

8/8 - 0s - loss: 0.0743 - accuracy: 0.9762 - 186ms/epoch - 23ms/step


In [19]:
prediction = (ann.predict(X_test_scaled)> 0.5).astype("int32")

In [20]:
results = pd.DataFrame({"predictions": prediction.ravel(), "actual": y_test})
results.head(10)

,predictions,actual
timestamp,,
2021-04-14,1,1.0
2020-04-13,1,1.0
2021-04-23,1,1.0
2019-08-09,0,0.0
2019-06-17,1,1.0
2017-11-20,0,0.0
2022-03-17,0,0.0
2017-11-02,1,1.0
2020-01-07,0,1.0


In [21]:
from sklearn import svm
from sklearn.metrics import classification_report

In [22]:
report = classification_report(y_test, prediction)


# Display the report
print(report)

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       143
         1.0       1.00      0.94      0.97       109

    accuracy                           0.98       252
   macro avg       0.98      0.97      0.98       252
weighted avg       0.98      0.98      0.98       252



In [52]:
tsla_df = pd.read_csv(filepath)
tsla_df["timestamp"] = pd.to_datetime(tsla_df["timestamp"]).dt.date
tsla_df = tsla_df.set_index("timestamp")
tsla_df.head()
tsla_df = tsla_df.iloc[1210:]
tsla_df_X = tsla_df[["Short_EWMA", "Long_EWMA", "macd", "signal_lag"]]
tsla_y = tsla_df[["Final_Signal"]]


In [53]:
tsla_prediction = (ann.predict(tsla_df_X)> 0.5).astype("int32")
tsla_prediction

array([[0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

In [54]:
report = classification_report(tsla_y, tsla_prediction)


# Display the report
print(report)

              precision    recall  f1-score   support

         0.0       0.87      1.00      0.93        39
         1.0       1.00      0.45      0.62        11

    accuracy                           0.88        50
   macro avg       0.93      0.73      0.78        50
weighted avg       0.90      0.88      0.86        50

